# 1. SETTINGS

In [1]:
# import packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
import scipy.stats
import os
import time

In [2]:
# helper functions
import functions
from functions import *

In [3]:
# pandas options
pd.set_option('display.max_columns', None)

In [4]:
# dark background style
plt.style.use('dark_background')

In [5]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [6]:
# garbage collection
import gc
gc.enable()

# 2. DATA PREPARATION

In [7]:
# import CSV
df = pd.read_csv('../data/data_v3.csv')
print(df.shape)

(500000, 13)


In [8]:
# target variable
target = 'fraud'

In [9]:
# partitioning
train = df[df[target].isnull() == False]
test  = df[df[target].isnull() == True]
print(train.shape)
print(test.shape)

(1879, 13)
(498121, 13)


In [10]:
# target variable
y = train.sort_values('id')[target]
test_ids = test['id']
del train, test

In [12]:
### IMPORT OOF PREDS

# which model to stack?
model = 'logit'

# profit threshold
min_profit = 250

# list names
names = sorted(os.listdir('../oof_preds'))
names = [n for n in names if int(n[n.rindex('_')+1:-4]) > min_profit]
names = [s for s in names if model in s]

# preprocessing loop
for name in names:

    # load preds
    tmp_tr = pd.read_csv('../oof_preds/'   + str(name))
    tmp_te = pd.read_csv('../submissions/' + str(name))

    # sort OOF preds by ID
    if 'id' in tmp_tr:
        tmp_tr = tmp_tr.sort_values('id')
        del tmp_tr['id']
    
    # rename columns
    tmp_tr.columns = [name]    
    tmp_te.columns = [name]    

    # cbind data
    if name == names[0]:     
        train = tmp_tr 
        test  = tmp_te
    else:
        train = pd.concat([train, tmp_tr], axis = 1)
        test  = pd.concat([test,  tmp_te], axis = 1)
        
# display information
print('- Train shape:', train.shape)
print('- Test shape:',  test.shape)

- Train shape: (1879, 4)
- Test shape: (498121, 4)


# 3. MODELING

In [13]:
# drop bad features
excluded_feats = ['id']
features = [f for f in train.columns if f not in excluded_feats]
print(train[features].shape)

(1879, 4)


In [14]:
### PARAMETERS

# settings
cores = 10
seed  = 999

# cross-validation
num_folds = 10
shuffle   = True

# muner of rounds
max_rounds = 400
stopping   = 400
verbose    = 100

# LGB parameters
lgb_params = {
    'boosting_type':     'gbdt',
    'objective':         'binary',
    'metrics':           'binary_logloss',
    'bagging_fraction':  0.9,
    'feature_fraction':  0.8,
    'lambda_l1':         0.1,
    'lambda_l2':         0.1,
    'min_split_gain':    0.01,
    'min_child_weight':  1,
    'min_child_samples': 1,
    'silent':            True,
    'verbosity':         -1,
    'learning_rate':     0.1,
    'max_depth':         7,
    'num_leaves':        70,
    'scale_pos_weight':  1,
    'n_estimators':      max_rounds,
    'nthread' :          cores,
    'random_state':      seed,
}

# data partitinoing
folds = StratifiedKFold(n_splits = num_folds, random_state = seed, shuffle = shuffle)

In [15]:
# placeholders
clfs = []
valid_profit = np.zeros(num_folds) 
preds_test   = np.zeros(test.shape[0])
preds_oof    = np.zeros(train.shape[0])
importances  = pd.DataFrame()

In [16]:
### CROSS-VALIDATION LOOP
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train, y)):
    
    # data partitioning
    trn_x, trn_y = train[features].iloc[trn_idx], y.iloc[trn_idx]
    val_x, val_y = train[features].iloc[val_idx], y.iloc[val_idx]
           
    # train lightGBM
    print('Custom early stopping: select the best out of %.0f iterations...' % max_rounds)
    clf = lgb.LGBMClassifier(**lgb_params) 
    clf = clf.fit(trn_x, trn_y, 
                  eval_set              = [(trn_x, trn_y), (val_x, val_y)], 
                  eval_metric           = prediction_reward, 
                  #eval_metric           = "logloss", 
                  #early_stopping_rounds = stopping,
                  verbose               = verbose)
    clfs.append(clf)
    
    # predict validation from the best iteration
    #best_iter = clf.best_iteration_
    best_iter = np.argmax(clf.evals_result_['valid_1']['profit']) + 1
    val_preds = clf.predict_proba(val_x, num_iteration = best_iter)[:, 1]
    print('Best iteration is:')
    print('[' + str(best_iter) + ']   valid_1 profit: ' + str(prediction_reward(val_y, val_preds)[1].astype('int')) + 
          "; log_loss = ", str(np.round(log_loss(val_y, val_preds), 6)))
       
    # predictions
    preds_oof[val_idx]    = clf.predict_proba(val_x, num_iteration = best_iter)[:, 1]
    valid_profit[n_fold]  = prediction_reward(val_y, preds_oof[val_idx])[1]
    preds_test           += clf.predict_proba(test[features], num_iteration = best_iter)[:, 1] / folds.n_splits 

    ## importance
    fold_importance_df = pd.DataFrame()
    fold_importance_df['Feature'] = features
    fold_importance_df['Importance'] = clf.feature_importances_
    fold_importance_df['Fold'] = n_fold + 1
    importances = pd.concat([importances, fold_importance_df], axis = 0)
    
    # print performance
    print('----------------------')
    print('FOLD%2d: PROFIT = %.0f' % (n_fold + 1, valid_profit[n_fold]))
    print('----------------------')
    print('')
        
    # clear memory
    del trn_x, trn_y, val_x, val_y
    gc.collect()
    
    
# print overall performance    
cv_perf = np.sum(valid_profit)
print('----------------------')
print('TOTAL PROFIT = %.0f' % cv_perf)
print('----------------------')

Custom early stopping: select the best out of 400 iterations...
[100]	training's binary_logloss: 0.0089096	training's profit: 335	valid_1's binary_logloss: 0.0213321	valid_1's profit: 5
[200]	training's binary_logloss: 0.00829733	training's profit: 400	valid_1's binary_logloss: 0.0203599	valid_1's profit: 30
[300]	training's binary_logloss: 0.00824127	training's profit: 400	valid_1's binary_logloss: 0.0203045	valid_1's profit: 30
[400]	training's binary_logloss: 0.00824127	training's profit: 400	valid_1's binary_logloss: 0.0203045	valid_1's profit: 30
Best iteration is:
[6]   valid_1 profit: 55; log_loss =  0.073958
----------------------
FOLD 1: PROFIT = 55
----------------------

Custom early stopping: select the best out of 400 iterations...
[100]	training's binary_logloss: 0.00879986	training's profit: 350	valid_1's binary_logloss: 0.0131874	valid_1's profit: 30
[200]	training's binary_logloss: 0.00829131	training's profit: 350	valid_1's binary_logloss: 0.0145126	valid_1's profit: 

In [17]:
##### RECHECK PROFIT  
prediction_reward(y, preds_oof)


###### TRACKING RESULTS (10 folds, start = True, seed = 999)

# V1: 3 logit models: 415 = 55 + 45 + 45 + 30 + 40 + 40 + 50 + 40 + 30 + 40

('profit', 415.0, True)

# 4. SUBMISSION

In [18]:
# file name
model = 'stack_logit'
perf  = str(round(cv_perf, 0).astype('int'))
name  = model + '_' + perf

In [19]:
# export OOF preds
oof = pd.DataFrame({'id': train.index, 'fraud': preds_oof})
oof.to_csv('../stage2_oof_preds/' + str(name) + '.csv', index = False)
oof.head()

,id,fraud
0,0,0.000534
1,1,0.030495
2,10,0.000480
3,100,0.022627
4,1000,0.022627


In [20]:
# check submission
sub = pd.DataFrame({'id': test_ids, 'fraud': preds_test})
#sub['fraud'] = np.round(sub['fraud']).astype('int')
sub.head()

,id,fraud
1879,unlab_0,0.020242
1880,unlab_1,0.020242
1881,unlab_2,0.020242
1882,unlab_3,0.020242
1883,unlab_4,0.020242


In [21]:
# export submission
sub = sub[['fraud']]
sub.to_csv('../submissions/' + str(name) + '.csv', index = False)
sub.shape

(498121, 1)

In [23]:
# check correlation with best individual submission
prev_sub = pd.read_csv('../submissions/lgb_v8_375.csv')
cor = np.sum(prev_sub[target] == np.round(sub.reset_index()[target])) / len(sub)
print("Share of the same predictions: " + str(np.round(cor, 6)))

Share of the same predictions: 0.985164
